In [ ]:
import torch
from fastai import *
from fastai.vision.all import *
import pandas as pd
import json

df = pd.read_csv('./cassava-leaf-disease-classification/train.csv')
path = Path('./cassava-leaf-disease-classification/train_images')
def get_x(r): return path/r['image_id']
def get_y(r): return str(r['label'])

In [ ]:
with open('./cassava-leaf-disease-classification/label_num_to_disease_map.json', 'r') as j:
     lbl_to_disease = json.loads(j.read())

In [ ]:
lbl_to_disease

In [ ]:
dblock = DataBlock(blocks=(ImageBlock, MultiCategoryBlock),
                   get_x = get_x, get_y = get_y, 
                   splitter=RandomSplitter(0.2),
                   item_tfms=RandomResizedCrop(224, min_scale=0.35),  
                   batch_tfms=aug_transforms(size=224, min_scale=0.75)) 
dsets = dblock.datasets(df)
dls = dblock.dataloaders(df)

In [ ]:
learn = cnn_learner(dls, resnet50, metrics=accuracy_multi)

In [ ]:
learn.fine_tune(3,freeze_epochs=4)

In [ ]:
lr_min, lr_steep = learn.lr_find()

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2,  slice(lr_min, lr_steep))

In [ ]:
path = '/home/aleksflames/Documents/notebooks/kaggle-leafy/cassava-leaf-disease-classification/test_images/'
def make_pred(path):
    lister = []
    files = os.listdir(path)
    for f in files:
        pred =learn.predict(path + f)
        tmp = {'image_id':f,
               'label':pred[0]}
        x = pd.DataFrame.from_dict([tmp])
        lister.append(x)
    df = pd.concat(lister)
    return df

In [ ]:
ret = make_pred()

In [ ]:
ret.to_excel('sample_submission.csv')